<a href="https://colab.research.google.com/github/Ilvecho/Project_6/blob/main/Project_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install TTS

from TTS.api import TTS
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import torch
from google.colab import files,drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Pre-trained TTS model

List of all available TTS pre-trained models in **English**

 - 'tts_models/en/ek1/tacotron2',
 - 'tts_models/en/ljspeech/tacotron2-DDC',
 - 'tts_models/en/ljspeech/tacotron2-DDC_ph',
 - 'tts_models/en/ljspeech/glow-tts',
 - 'tts_models/en/ljspeech/speedy-speech',
 - 'tts_models/en/ljspeech/tacotron2-DCA',
 - 'tts_models/en/ljspeech/vits',
 - 'tts_models/en/ljspeech/vits--neon',
 - 'tts_models/en/ljspeech/fast_pitch',
 - 'tts_models/en/ljspeech/overflow',
 - 'tts_models/en/ljspeech/neural_hmm',
 - 'tts_models/en/vctk/vits',
 - 'tts_models/en/vctk/fast_pitch',
 - 'tts_models/en/sam/tacotron-DDC',
 - 'tts_models/en/blizzard2013/capacitron-t2-c50',
 - 'tts_models/en/blizzard2013/capacitron-t2-c150_v2',
 - 'tts_models/en/multi-dataset/tortoise-v2',

In [ ]:
from TTS.api import TTS

device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS('tts_models/multilingual/multi-dataset/your_tts').to(device)

In [24]:
# generate speech by cloning a voice using default settings
tts.tts_to_file(text="Let's go back to English, apparently the multi-lingual model to output Italian does not work that well. In any case, in this project we need to use English, so it's not a big deal",
                speaker_wav="/content/gdrive/MyDrive/P6_files/Test3.wav",
                language='en',
                file_path="/content/gdrive/MyDrive/P6_files/output.wav")

 > Text splitted to sentences.
["Let's go back to English, apparently the multi-lingual model to output Italian does not work that well.", "In any case, in this project we need to use English, so it's not a big deal"]
 > Processing time: 2.0741336345672607
 > Real-time factor: 0.204428704372882


'/content/gdrive/MyDrive/P6_files/output.wav'

Import the Pre-Trained Tacotron2 model from NVIDIA

In [3]:
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn

Import the pre-trained model WaveGlow from NVIDIA

In [4]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Get the tool to properly encode the Text and use it on a sample text.

Note that in this case we are using a **character-based encoding**

In [10]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence(["Hello World, I am a machine ready to take over!"])

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [11]:
sequences

tensor([[45, 42, 49, 49, 52, 11, 60, 52, 55, 49, 41,  6, 11, 46, 11, 38, 50, 11,
         38, 11, 50, 38, 40, 45, 46, 51, 42, 11, 55, 42, 38, 41, 62, 11, 57, 52,
         11, 57, 38, 48, 42, 11, 52, 59, 42, 55,  2]], device='cuda:0')

From the sample text generate the Mel Spectrogram with tacotron2 and then convert the Mel in wave audio using WaveGlow

In [8]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [9]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)